# Elasto-visco-plastic damage model with temperature evolution- 1D interface

**Authors:** RC, MA, MV, AB<br>
**Date Created:** 20/02/2024  (AB)<br>
**Last Modified:** 28/02/2024 (AB)<br>
**Description:** Pilot implementation of the coupled elasto-visco-plastic damage model of an interface able to capture fatigue and creep in a consistent way with extension to capture temperature evolution.

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import sympy as sp

Model formulation focusing on the new added terms related to temperature

Helmholtz free energy:
\begin{equation} \label{eq:helmhltz_free_enrg}
\rho \psi (s, s^\mathrm{vp}, \omega, \alpha, z, T) = \frac{1}{2} (1 - \omega)  E_\mathrm{b}  (s - s^\mathrm{vp})^2 + \frac{1}{2} \gamma \alpha ^2 + \frac{1}{2} K z^2 + g_1(T) 
\end{equation}

With 
\begin{equation}
 g_1(T)  = \frac{ C_v \cdot T^2}{2 T_0}
\end{equation}

where $T$ is the temperature, $C_v$ is the specific heat, and $T_0$ is the intitial temperature.

Thermodynamic forces:

The entropy can be given as 
\begin{equation} 
 \bar{S} = \frac{\partial \rho  \psi}{\partial T} = {g_1}^{'}(T) = \dfrac{C_v  \cdot T}{T_0} 
\end{equation}

The threshold function remains unchanged

\begin{equation} \label{eq:flow_potential}
\phi = f(\tilde{\tau}^{\pi}, X, Z) + \frac{S (1-\omega)^c  }{(r+1)} \left(\frac{Y}{S}\right) ^{r+1} \cdot g_2 (T) + g_3 (\bar{S}),
\end{equation}


where $g_2 (T)$ is a function introducing amplication factor on the damage evolution depending on temperature $T$ and controlld with material parameter $\bar{T}$
\begin{equation} 
 g_2 (T)  =  \left( 1 + \bar{T} \sqrt{T}  \right) 
\end{equation}

and $g_3 (\bar{S})$ is the term allows to obtain the temperature evolution in similar way to damage evolution
\begin{equation} 
 g_3 (\bar{S})  = \frac{\bar{S}}{S_\mathrm{t}} 
\end{equation}

$S_\mathrm{t}$ is a material paramter.

The temperature evolution can be written as
\begin{equation}
\dot{T} = \frac{\partial \phi}{\partial \bar{S}} = \dot{\lambda} \cdot \frac{1}{S_\mathrm{t}}
\end{equation}




## State update method

This is a function `get_state_n1` to calculate the next state of a viscoplastic model. The function takes as input the variables `s_n, ds, dt, Eps_n, tau_pi_bar, K, gamma, E_b, S, c, r, m, sigma_n, eta, S_t, T_bar` and returns the next state.

Function input parameters:

- `s_n`: current scalar variable representing the state at the current time step,
- `ds`:  change in the state variable in the current time step,
- `dt`: time step,
- `Eps_n`: list of internal variables at the current time step,
- `tau_pi_bar, K, gamma, E_b, S, c, r, m, sigma_n, eta, S_t, T_bar`: material parameters. 

Steps of the function:

1. The function first calculates the trial stress (`tau_trial`) and the trial yield function (`f_trial`).
2. If the yield condition is satisfied (`f_trial > 1e-12`), the algorithm performs a return mapping:
     - It first calculates the consistency parameter `delta_lambda`.
     - Then, it updates the state variables considering plastic deformation – `s_vp_n1`, `Y_n1`, `w_p`, `alpha_p`, `z_p`, `tau_p`.
3. This implementation includes a regularization using a visco-plastic approach, where the interpretation could be seen as a mechanism to capture the transient effects of the visco-plastic processes. The function calculates the relaxed time `relax_t` which substantially depends on the effective stiffness and not on the initial stiffness.
4. The overshoot stress is calculated considering the effective stiffness and not the initial stiffness.
5. The internal state variables `tau_vp_n1`, `z_vp_n1` and `alpha_vp_n1` are updated with regularization.
     - The function also considers the regularization of damage `w_vp_n1`, which theoretically could be interpreted as changes in the microstructure.
6. If the yield condition is not satisfied, the algorithm treats the response as elastic and simply returns the same internal variables that were input.

Please note that this function assumes a perfect coupling between damage and viscoplasticity: the visco-plastic parameters are the same as the ones for plasticity and are affected by damage in a similar way. Changes in the parameters can be made to reflect different coupling behaviors, if necessary.

In [ ]:
import numpy as np


def get_state_n1(s_n, ds, dt, Eps_n, tau_pi_bar, K, gamma, E_b, S, c, r, m, sigma_n, eta, S_t, T_bar):
    w_vp_n, s_vp_n, z_vp_n, alpha_vp_n, tau_vp_n, temp_vp_n = Eps_n

    # Here I made relaxation time independent of damage (to be further discussed)
    relax_t = eta / (E_b + K + gamma)

    s_n1 = s_n + ds
    tau_trial = (1 - w_vp_n) * E_b * (s_n1 - s_vp_n)
    f_trial = np.abs(tau_trial / (1 - w_vp_n) - gamma * alpha_vp_n) - tau_pi_bar - K * z_vp_n + m * sigma_n

    if f_trial > 1e-12:  # plastic

        # Return mapping
        delta_lambda = f_trial / (E_b / (1 - w_vp_n) + gamma + K)
        # update all the state variables
        s_vp_n1 = (s_vp_n + delta_lambda * 
                   np.sign(tau_trial/(1. - w_vp_n) - gamma * alpha_vp_n) / (1 - w_vp_n))
        
        # Evolution of temperature
        temp_p_n1 =  temp_vp_n + (delta_lambda*(1./S_t))
        # prepare the term introducing the effect of temperature on damage evoultion
        temp_eff = (1 + T_bar * np.sqrt(temp_p_n1))
        
        Y_n1 = 0.5 * E_b * (s_n1 - s_vp_n) ** 2
        w_p = w_vp_n + ((1 - w_vp_n) ** c) * (delta_lambda * (Y_n1 / S) ** r)  * (1 - sigma_n / (0.5 * tau_pi_bar)) * temp_eff
        alpha_p = alpha_vp_n +  delta_lambda * np.sign(tau_trial/(1 - w_vp_n) - gamma * alpha_vp_n)
        z_p = z_vp_n + delta_lambda
        tau_p = E_b * (1. - w_p) * (s_n1 - s_vp_n1)
    
        # visco-plastic regularization

        dt_tau = dt / relax_t
        # Another modification has been done here to calculate the 
        # overshoot stress considering the effective stiffness and 
        # not the initial stiffness
        tau_vp_n_ds = tau_vp_n + (1 - w_p) * E_b * ds
        tau_vp_n1 = (tau_vp_n_ds + dt_tau * tau_p)/(1 + dt_tau)
        z_vp_n1 = (z_vp_n + dt_tau * z_p)/(1 + dt_tau) 
        alpha_vp_n1 = (alpha_vp_n + dt_tau * alpha_p)/(1 + dt_tau) 

        # Does it make sense to regularize the damage as well?
        # what would be the interpretation in terms of changes in the 
        # microstructure?
        # w_vp_n1 = (w_vp_n + dt_tau * w_p)/(1 + dt_tau) 
        w_vp_n1 = w_p
        
        # Here I am using the effective stiffness to calculate the viscoplastic strain which I think it is necessary. 
        s_vp_n1 = s_n1 - tau_vp_n1/((1 - w_p)*E_b)
        
        # Similar to damage (no viscoplastic regularization?)
        temp_vp_n1 =  temp_p_n1 


        return w_vp_n1, s_vp_n1, z_vp_n1, alpha_vp_n1, tau_vp_n1, temp_vp_n1

    else:  # elastic
        return w_vp_n, s_vp_n, z_vp_n, alpha_vp_n, tau_trial, temp_vp_n


## Time-stepping algorithm

Time integration loop for a viscoplastic model based on the backward Euler scheme.

The `time_stepping` function simulates the material response to some load over a series of time steps. Each time step represents a change in internal state, and with each new time step, the function updates the accumulated values of the considered variables.

Inputs to the function are:

- `s_max`: the maximum stress state to be reached in the simulation,
- `n_steps`: the number of increments used to reach `s_max`,
- `dt`: the time step length.

Steps of the function:

1. This function firstly creates an empty list for every variable it is going to update through the simulation. These lists store the value of each of these variables at every time step.

2. It then begins a loop over `n_steps`, wherein it calculates the increment of strain (`ds`) at each step and gets the values of the viscoplastic variables at the next state (`Eps_n1`).

3. `Eps_n1` is then used to update the previously empty lists that store the values of these variables (the state variables, stress, and damage).

4. Finally, once all steps are over, the function returns these lists (i.e., the full history of each variable) along with the time steps (`t_arr`).

This function allows us to simulate the material response under a load gradually applied over time until it reaches `s_max`. Furthermore, since viscoplasticity is considered, the function is effective in simulating cases where creep or relaxation happen over the series of time steps.

Remember that in line with our previous assumptions in `get_state_n1`, in the `time_stepping` function we consider a perfect coupling between the damage and viscoplastic responses with the same parameters for both responses.

In [ ]:

def time_stepping(s_arr, t_arr, **material_params):
    ds_arr = np.diff(s_arr,)
    dt_arr = np.diff(t_arr)
    Eps_arr = np.zeros((len(s_arr), 6))

    for n, dt in enumerate(dt_arr):
        ds = ds_arr[n]
        s_n = s_arr[n]
        Eps_arr[n+1] = get_state_n1(s_n, ds, dt, Eps_arr[n], **material_params)

    return s_arr, t_arr, Eps_arr.T

# Auxiliary methods

Plotting functions for the verification studies below

## Load functions

### Step load function

This code defines a symbolic step function `fn_s_t` using the Sympy Python library. Symbols (`s_1`, `t`, and `t_1`) are first defined, which represent slip, time, and a specific time point respectively.

The `Piecewise` function is then used to define `fn_s_t`, which is a step function of the input time `t`. This function behaves as follows:
- If `t` (the time) is less than `t_1` (a specific time point), the function output is a straight line with the slope `s_1 / t_1`. This part represents the linear increasing phase of slip, the slope `s_1 / t_1` is the rate at which slip increases per unit time.
- If `t` (the time) is equal to or greater than `t_1`, the function output is a constant value `s_1`. This part represents the constant phase of slip after the loading reaches its peak value `s_1`.

Then, `sp.lambdify` is used to convert the symbolic function `fn_s_t` into a lambda function `get_step_loading` that can be used with numerical computation. The lambda function takes inputs of time `t`, maximum slip `s_1` and specific point of time `t_1`.

Finally, `fn_s_t` is printed out.

In general, this block of code is handy for modelling situations where a value increases linearly over time until a specific point, before staying constant.

In [ ]:
s_1 = sp.Symbol('s_1')
t_1 = sp.Symbol('t_1')
t = sp.Symbol('t')
fn_s_t = sp.Piecewise((t * s_1/t_1, t < t_1),(s_1, True)
)
get_step_loading = sp.lambdify((t, s_1, t_1), fn_s_t)
fn_s_t

### Cyclic loading function 

`get_cyclic_load` is a function that simulates a loading history for a cyclic load. The function accepts the maximum slip `max_s`, total simulating time `max_t`, the number of loading cycles `n_cycles`, and the number of increments `n_incr`.

In the function:

1. `np.tile(np.array([-1, 1]), n_cycles)` generates an array that alternates between -1 and 1 for `n_cycles` times. This represents the loading and unloading actions.
2. It then multiplies this array by `np.linspace(0, max_s, 2 * n_cycles)`, which simply creates an array of `2 * n_cycles` evenly-spaced values from 0 to `max_s`. This construct results in a loading history (`s_arr`) that progressively increases in absolute magnitude each cycle, from 0 to `max_s`.
3. Subsequently, `s_arr` is interpolated over a new set of time points. `np.linspace(0, max_t, n_incr * len(s_arr))` creates a list of evenly spaced time points over the total time with more time intervals. The interpolation refines the loading history `s_arr` to higher resolution.

The function also generates a time array (`t_arr`) with the same length as the interpolated `s_arr`. This array represents the simulation time points.

Finally, the function returns the loading history (`s_arr`) and the time array (`t_arr`), which could be used for analysis or simulation.

In [ ]:
def get_cyclic_load(max_s, max_t, n_cycles, n_incr):
    # Generating loading history
    s_arr = np.tile(np.array([-1, 1]), n_cycles) * np.linspace(0, max_s, 2 * n_cycles)
    s_arr = np.interp(np.linspace(0, max_t, n_incr * len(s_arr)), np.linspace(0, max_t, len(s_arr)), s_arr)

    # time array as input
    t_arr = np.linspace(0, max_t, len(s_arr))
    return s_arr, t_arr

### Plotting response

The function `plot_response` is used for visualizing the simulation results. This function accepts four arguments: `param_name`, `s_arr` (array of slip), `t_arr` (array of companion timestamps), and `response_values` (dictionary containing responses).

Four subplots are generated using this function:
1. **Loading scenario**: This plot displays how slip changes with time for different values of the parameter specified by `param_name`. This information is overlaid with the viscoplastic stress (`tau_vp`) change over time.
2. **Stress-Slip relation**: This plot illustrates how viscoplastic stress (`tau_vp`) changes with slip for different parameter values.
3. **Evolution of viscoplastic slip**: This plot displays how viscoplastic slip (`s_vp`) evolves with total slip for different parameter values.
4. **Damage evolution**: This shows how the scalar damage variable (`w`) evolves with slip over time for different parameter values.

Each subplot includes a legend depicting the parameter name and its corresponding value, and zero lines for easy reference.

In [ ]:
def plot_response(param_name, response_values):
    fig, ((ax1,  ax2), (ax3,  ax4)) = plt.subplots(2,2, tight_layout=True, figsize=(7,7))
    fig.canvas.header_visible = False
    ax1_twin = ax1.twinx()

    for (param, rv), color in zip(response_values.items(), ['black', 'red', 'green']):
        s_arr, t_arr, rp = rv
        w, s_vp, z_vp, alpha_vp, tau_vp, temp_vp = rp
        ax1.plot(t_arr, s_arr, color=color, linewidth=1, label="{} = {}".format(param_name, param))  # Loading scenario
        ax1_twin.plot(t_arr, tau_vp, linestyle='dashed', color=color, linewidth=1)
        ax2.plot(s_arr, tau_vp, color=color, linewidth=1, label="{} = {}".format(param_name, param))    # Stress-slip relation
        ax3.plot(t_arr, temp_vp, color=color, linewidth=1, label="{} = {}".format(param_name, param))      # Evolution of temp
        ax4.plot(s_arr, w, color=color, linewidth=1, label="{} = {}".format(param_name, param))         # Damage evolution

    ax1.axhline(y=0, color='k', linewidth=1, alpha=0.5)
    ax1.axvline(x=0, color='k', linewidth=1, alpha=0.5)
    ax1.set_title('loading scenario')
    ax1.set_xlabel('time [s]')
    ax1.set_ylabel('slip [mm]')
    ax1.legend()

    ax2.axhline(y=0, color='k', linewidth=1, alpha=0.5)
    ax2.axvline(x=0, color='k', linewidth=1, alpha=0.5)
    ax2.set_title('stress-slip')
    ax2.set_xlabel('slip [mm]')
    ax2.set_ylabel('stress [MPa]')
    ax2.legend()

    ax3.axhline(y=0, color='k', linewidth=1, alpha=0.5)
    ax3.axvline(x=0, color='k', linewidth=1, alpha=0.5)
    ax3.set_title('evolution of temperature')
    ax3.set_xlabel('time [sec]')
    ax3.set_ylabel('temperature [$^{\circ}$C]')
    ax3.legend()

    ax4.axhline(y=0, color='k', linewidth=1, alpha=0.5)
    ax4.axvline(x=0, color='k', linewidth=1, alpha=0.5)
    ax4.set_title('damage evolution')
    ax4.set_xlabel('slip [mm]')
    ax4.set_ylabel('damage [-]')
    ax4.legend()

# Verification studies

Run elementary verification studies for 
 - a monotonically increasing slip loading
 - step loading representing the relaxation test
 - cyclic loading with an increasing amplitude

## Load rate test

Test the effect of the loading rate with constant value of $\eta$ and vary the loading rate $\dot{s}$.

The code block is performing a simulation of a system response under various rates of slip (`dot_s`).

`t_max` and `n_t` specify the maximum time and number of time points for the simulation. `t_arr` is an array of linearly distributed time points from 0 to `t_max`.

For each value of the slip rate (`dot_s`), it calculates a slip (`s_arr`) at each time point in `t_arr` by multiplying the entire `t_arr` with `dot_s`.

Then, for each slip rate, the code stores the simulation result in the dictionary `response_values` by calling the `time_stepping` function with `s_arr`, `t_arr`, and some parameters.

In [ ]:
s_max, n_t = 50, 1000
response_values = {}
for dot_s in 0.01, 0.1, 1:
    t_max = s_max / dot_s
    t_arr = np.linspace(0, t_max, n_t)
    s_arr = dot_s * t_arr
    response_values[dot_s] = time_stepping(
        s_arr, t_arr, tau_pi_bar=1, K=0, gamma=0, E_b=100, S=50, c=1, r=1, 
        m=0, sigma_n=0, eta=10, S_t = 1e+3, T_bar =0)

plot_response(r'$\dot{s}$', response_values)

## Relaxation test

Reproduce the analytical solution of step loading with the decay of stress towards the time-independent solution

In [ ]:
t_max, n_t = 6, 100
t_arr = np.linspace(0, t_max, n_t+1)
s_arr = get_step_loading(t_arr, s_1=0.12, t_1=1)

In [ ]:
response_values = {}
for eta in [1, 10, 100]:
    response_values[eta] = time_stepping(
        s_arr, t_arr, tau_pi_bar=1, K=0, gamma=0, 
        E_b=100, S=1e+1, c=2, r=1, 
        m=0, sigma_n=0, eta=eta, S_t = 1e+3, 
        T_bar =0)

plot_response('eta', response_values)

## Cyclic loading test

Test the effect of viscosity for a cyclic loading with increasing amplitude

### Parametric study of viscosity parameter $\eta$

In [ ]:
s_arr, t_arr = get_cyclic_load(
    n_cycles = 3, max_s = 50, max_t = 10, n_incr = 100
)
response_values = {}
for eta in 0.1, 2, 10:
    response_values[eta] = time_stepping(
        s_arr, t_arr, tau_pi_bar=1, K=0, gamma=0, E_b=100, S=1e+5, c=1, r=1, 
        m=0, sigma_n=0, eta=eta, S_t = 1e+3, T_bar =0)
    
plot_response(r'$eta$', response_values)

### Parametric study of loading rate influence on temperature evolution

In [ ]:
response_values = {}
for dot_s in 0.05, 0.1, 1:
    max_s = 10
    s_arr, t_arr = get_cyclic_load(
    n_cycles = 15, max_s = 10, max_t = max_s/dot_s, n_incr = 100)
    
    response_values[dot_s] = time_stepping(
        s_arr, t_arr, tau_pi_bar=1, K=0, gamma=0, E_b=100, S=1e+5, c=1, r=1, 
        m=0, sigma_n=0, eta=eta, S_t = 2e+2, T_bar =0)
    
plot_response(r'$\dot{s}$', response_values)

### Parametric study of the parameter T_bar (controling the influence od temperature on damage evolution)

In [ ]:
response_values = {}
for T_bar in 0, 1 ,10:
    max_s = 10
    dot_s = 1
    s_arr, t_arr = get_cyclic_load(
    n_cycles = 10, max_s = 3, max_t = max_s/dot_s, n_incr = 100)
    response_values[T_bar] = time_stepping(
        s_arr, t_arr, tau_pi_bar=1, K=0, gamma=0, E_b=100, S=1e+5, c=1, r=1, 
        m=0, sigma_n=0, eta=eta, S_t = 5e+2, T_bar =T_bar)
    
plot_response(r'$\bar{T}$', response_values)